# Algorithm Explained

In [ ]:
import sys
sys.path.append("./hybridjobs/utility")

from hybridjobs.utility.ProteinParser import ProteinData
from hybridjobs.utility.ProteinModel import ProteinModel
from hybridjobs.utility.ProteinStructurePrediction import ProteinStructurePrediction
# from utility.ResultProcess import ResultParser
import time

timestamp = time.strftime("%Y%m%d-%H")

Using TensorFlow backend.


#### Step 1: Prepare Data

In this part, we load the folder with the raw rna data for experimentation. To evaluate the quantum solution and the actual solution include both fasta and ct files. To only generate a quantum solution, upload a fasta file.

In [2]:
# input: aminoacids
# output: energy files

protein_name = 'glycylglycine'
aminoacids = 'GG'
number_bits_to_discretize_protein_angles = 4
protein_id = 0

input_filename = "inputRotations"
output_filename = "outputRotations"
basis = "6-31g"
energy_method = "mp2"



In [3]:
data_path='protein-folding-data'

#### Step 2: Build Model

In this part, we will show how to build model for qfold

In [4]:
# initial the ProteinFold object
init_param = {}
# method: qfold-cc stands for the classical metropolis method in QFold
# method: qfold-qc stands for the quantum metropolis method in QFold
method = ['qfold-cc', 'qfold-qc']

for mt in method:
    if mt == 'qfold-cc':
        init_param[mt] = {}
        init_param[mt]['params'] = ["initialization"]
    elif mt == 'qfold-qc':
        init_param[mt] = {}
        init_param[mt]['params'] = ["initialization"]

config_path = "ProteinFoldingHybridJobs/config/config.json"
protein_model = ProteinModel(data_path, method, config_path, **init_param)

INFO:root:Initial parameters for protein glycylglycine_3_GG using qfold-cc
INFO:root:Initial parameters for protein glycylglycine_3_GG using qfold-qc
INFO:root:Initial parameters for protein glycylglycine_4_GG using qfold-cc
INFO:root:Initial parameters for protein glycylglycine_4_GG using qfold-qc


In [5]:
# set the parameters for model
model_param = {}

method = 'qfold-cc'
model_param[method] = {}

# parameters
model_param[method]['initialization'] = ["minifold", "random"]

method = 'qfold-qc'
model_param[method] = {}

# parameters
model_param[method]['initialization'] = ["minifold", "random"]

protein_model.build_models(**model_param)


deltas_dict length for glycylglycine_3_GG: 256
deltas_dict length for glycylglycine_3_GG: 256
deltas_dict length for glycylglycine_4_GG: 1024
deltas_dict length for glycylglycine_4_GG: 1024
deltas_dict length for glycylglycine_3_GG: 256
deltas_dict length for glycylglycine_3_GG: 256
deltas_dict length for glycylglycine_4_GG: 1024
deltas_dict length for glycylglycine_4_GG: 1024


0

In [6]:
# save the model
model_path = protein_model.save("latest")

print(f"You have built the protein folding models and saved them as protein_folding_latest.pickle")

INFO:root:finish save protein_folding_latest.pickle


You have built the protein folding models and saved them as protein_folding_latest.pickle


#### Step 3: Predict Protein Structure

In this part, we will show how to run models for predicting protein structure

In [7]:
protein_models = ProteinModel.load(model_path)

In [8]:
model_info = protein_models.describe_models()

INFO:root:debug describe
INFO:root:model name: glycylglycine_3_GG, method: qfold-cc
INFO:root:param: initialization, value {'minifold', 'random'}
INFO:root:model name: glycylglycine_3_GG, method: qfold-qc
INFO:root:param: initialization, value {'minifold', 'random'}
INFO:root:model name: glycylglycine_4_GG, method: qfold-cc
INFO:root:param: initialization, value {'minifold', 'random'}
INFO:root:model name: glycylglycine_4_GG, method: qfold-qc
INFO:root:param: initialization, value {'minifold', 'random'}


In [9]:
# get the model you want to optimize
protein_name = 'glycylglycine_3_GG'
initialization = 'random'
method = 'qfold-cc'

model_name = "{}+{}".format(protein_name, initialization)

protein_model = protein_models.get_model(protein_name, method, model_name)



In [10]:
data_path = 'data'
# psp_param stands for the parameters for predicting protein structure
psp_param = {}
psp_param["data_path"] = data_path
psp_param["mode"] = 'local-simulator'
psp_param["model_name"] = model_name
psp_param["model_path"] = model_path

psp = ProteinStructurePrediction(protein_model, method, config_path, **psp_param)

psp.run()



INFO:root:initial protein structure prediction using qfold-cc in QFold
INFO:root:CLASSICAL METROPOLIS: Time for 2 steps: 0.3307058811187744 seconds
INFO:root:CLASSICAL METROPOLIS: Time for 3 steps: 0.46265339851379395 seconds
INFO:root:CLASSICAL METROPOLIS: Time for 4 steps: 0.6084005832672119 seconds
INFO:root:CLASSICAL METROPOLIS: Time for 5 steps: 0.7418649196624756 seconds
INFO:root:CLASSICAL METROPOLIS: Time for 6 steps: 0.8906919956207275 seconds
INFO:root:CLASSICAL METROPOLIS: Time for 7 steps: 1.020871877670288 seconds
INFO:root:CLASSICAL METROPOLIS: Time for 8 steps: 1.1487469673156738 seconds
INFO:root:CLASSICAL METROPOLIS: Time for 9 steps: 1.298386812210083 seconds
INFO:root:finish save tts_results_glycylglycine_3_GG+random_1000_qfold-cc.json


In [11]:
initialization = 'random'
method = 'qfold-qc'

model_name = "{}+{}".format(protein_name, initialization)

protein_model = protein_models.get_model(protein_name, method, model_name)



In [12]:
psp = ProteinStructurePrediction(protein_model, method, config_path, **psp_param)

psp.run()



INFO:root:initial protein structure prediction using qfold-qc in QFold
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.10872 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05507 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.04888 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: UnitarySynthesis - 0.00954 (ms)
INFO:qiskit.transpiler.runningpassmanager:Pass: UnrollCustomDefinitions - 28044.69824 (ms)
INFO:qiskit.transpiler.passes.basis.basis_translator:Begin BasisTranslator from source basis {('mcx', 4), ('mcu1', 9), ('mcx', 5), ('snapshot', 16), ('cx', 2), ('ccx', 3), ('cu3', 2), ('x', 1), ('h', 1)} to target basis {'cy', 'mcz', 'mcry', 'set_statevector', 'u2', 'ry', 'sdg', 'quantum_channel', 'kraus', 'x', 'cu', 'initialize', 'rz', 'mcx', 'mcy', 'save_state', 'mcphase', 'roerror', 'rzz', 'cx', 'cz', 'ryy', 'r', 'tdg', 'mcu3', 'mcrz', 'unitary', 'save_probs_ket', 'save_statevector', 'h', 'save_probs', 'u3', 'pauli', 'save_expval', 'cu2', 'csx', 'sx',

#### Step 4: Post Process

In this part, we will show how to visualize the results

# Hybrid Job Experiment